In [1]:
import tensorflow as tf
import importlib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import torch
import utils4
import model2
importlib.reload(utils4)
importlib.reload(model2)
from utils4 import *
from model2 import *
tf.test.is_built_with_cuda()

True

In [2]:
icon_name_category, sketch_name_category = get_icons_and_sketches()
icon_dictionary, sketch_dictionary = load_icons_sketches_dic(icon_name_category, sketch_name_category)



In [3]:
icon_categories =  np.unique(icon_name_category[:, 1], return_index=False)
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(icon_categories)

# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
icon_categories_dic = {}
for i in range(len(icon_categories)):
    icon_categories_dic[icon_categories[i]] = onehot_encoded[i]

len_data = len(icon_name_category)
p_train=0.9
p_test=0.1
num_train = int(np.ceil(len_data*p_train))
num_test = int(np.floor(len_data*p_test))

icon_name_category = shuffle(icon_name_category)
icons_Train = icon_name_category[:num_train]
icons_Test= icon_name_category[-num_test:]

print(f'We have {len(icons_Train)} samples in the training set.')
print(f'We have {len(icons_Test)} samples in the test set.')

We have 1061 samples in the training set.
We have 117 samples in the test set.


In [31]:
BATCH_SIZE = 128
num_epochs = 50
optimizer = tf.keras.optimizers.Adam(0.001)
tf.keras.backend.set_floatx('float32')
iconClassificationModel = mynet(len(icon_categories))
loss = compute_cross_entropy
def train_step(icons, targets):
    with tf.GradientTape() as tape:
        outputs = iconClassificationModel(icons, training = True, model = "Classification")   
        tape.watch(outputs)
        current_loss = loss(outputs, targets)
    grads = tape.gradient(current_loss, iconClassificationModel.trainable_variables)
    optimizer.apply_gradients(zip(grads, iconClassificationModel.trainable_variables))
    return current_loss

In [32]:
for epoch in range(num_epochs):
    epoch_loss_avg = tf.keras.metrics.Mean()
    
    for i in range(0, len(icon_name_category), BATCH_SIZE):
        icon_name_category_batch = icon_name_category[i:i+BATCH_SIZE]
        icons, targets = get_batch_icons_targets(icon_name_category_batch, icon_dictionary, icon_categories_dic)
        loss_value = train_step(icons, targets)
        epoch_loss_avg.update_state(loss_value)
    print("Epoch {:d}: Loss: {:.3f}".format(epoch,epoch_loss_avg.result()))
    acc = 0
    for i in range(0, len(icon_name_category), BATCH_SIZE):
        icon_name_category_batch = icon_name_category[i:i+BATCH_SIZE]
        icons, targets = get_batch_icons_targets(icon_name_category_batch, icon_dictionary, icon_categories_dic)
        outputs = iconClassificationModel(icons, training = False, model = "Classification")
        correct_prediction = tf.equal(tf.argmax(outputs,1),tf.argmax(targets,1))
        for prediction in correct_prediction:
            if prediction:
                acc = acc + 1
    print("Test accuracy is: " + str(acc/len(icon_name_category)))
#iconClassificationModel.save_weights('Training-Weights/iconClassification')


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 0: Loss: 4

In [25]:
acc = 0
for i in range(0, len(icons_Test), BATCH_SIZE):
    icon_name_category_batch = icons_Test[i:i+BATCH_SIZE]
    icons, targets = get_batch_icons_targets(icon_name_category_batch, icon_dictionary, icon_categories_dic)
    outputs = iconClassificationModel(icons, training = False, model = "Classification")
    correct_prediction = tf.equal(tf.argmax(outputs,1),tf.argmax(targets,1))
    for prediction in correct_prediction:
        if prediction:
            acc = acc + 1
    #print(correct_prediction)
print("Test accuracy is: " + str(acc/len(icons_Test)))

Test accuracy is: 0.1282051282051282


In [24]:
iconClassificationModel.load_weights('Training-Weights/iconClassification')

## Sketch Classification

In [3]:
sketch_categories =  np.unique(sketch_name_category[:, 1], return_index=False)
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(sketch_categories)

# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
sketch_categories_dic = {}
for i in range(len(sketch_categories)):
    sketch_categories_dic[sketch_categories[i]] = onehot_encoded[i]

len_data = len(sketch_name_category)
p_train=0.9
p_test=0.1
num_train = int(np.ceil(len_data*p_train))
num_test = int(np.floor(len_data*p_test))

sketch_name_category = shuffle(sketch_name_category)
sketches_Train = sketch_name_category[:num_train]
sketches_Test= sketch_name_category[-num_test:]

print(f'We have {len(sketches_Train)} samples in the training set.')
print(f'We have {len(sketches_Test)} samples in the test set.')

We have 28626 samples in the training set.
We have 3180 samples in the test set.


In [46]:
BATCH_SIZE = 128
num_epochs = 100
optimizer = tf.keras.optimizers.Adam(0.001)
tf.keras.backend.set_floatx('float32')
sketchClassificationModel = mynet(len(sketch_categories))
loss = compute_cross_entropy
def train_step(sketches, targets):
    with tf.GradientTape() as tape:
        outputs = sketchClassificationModel(sketches, training = True, model = "Classification")   
        tape.watch(outputs)
        current_loss = loss(outputs, targets)
    grads = tape.gradient(current_loss, sketchClassificationModel.trainable_variables)
    optimizer.apply_gradients(zip(grads, sketchClassificationModel.trainable_variables))
    return current_loss

In [47]:
for epoch in range(num_epochs):
    epoch_loss_avg = tf.keras.metrics.Mean()
    sketches_Train = shuffle(sketches_Train)
    for i in range(0, len(sketches_Train), BATCH_SIZE):
        sketch_name_category_batch = sketches_Train[i:i+BATCH_SIZE]
        sketches, targets = get_batch_sketches_targets(sketch_name_category_batch, sketch_dictionary, sketch_categories_dic)
        loss_value = train_step(sketches, targets)
        epoch_loss_avg.update_state(loss_value)
    print("Epoch {:d}: Loss: {:.3f}".format(epoch,epoch_loss_avg.result()))
    acc = 0
    for i in range(0, len(sketches_Test), BATCH_SIZE):
        sketch_name_category_batch = sketches_Test[i:i+BATCH_SIZE]
        sketches, targets = get_batch_sketches_targets(sketch_name_category_batch, sketch_dictionary, sketch_categories_dic)
        outputs = sketchClassificationModel(sketches, training = False, model = "Classification")
        correct_prediction = tf.equal(tf.argmax(outputs,1),tf.argmax(targets,1))
        for prediction in correct_prediction:
            if prediction:
                acc = acc + 1
    print("Test accuracy is: " + str(acc/len(sketches_Test)))
#iconClassificationModel.save_weights('Training-Weights/iconClassification')


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 0: Loss: 3

In [48]:
sketchClassificationModel.save_weights('Training-Weights/sketchClassification2')